In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data transformations (CIFAR-specific normalization)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

# Function to create dataloaders
def get_dataloader(dataset, batch_size, train):
    data = dataset(root='./data', train=train, download=True, transform=transform)
    return DataLoader(data, batch_size=batch_size, shuffle=train)

train_loader_10 = get_dataloader(datasets.CIFAR10, 128, train=True)
test_loader_10 = get_dataloader(datasets.CIFAR10, 128, train=False)
train_loader_100 = get_dataloader(datasets.CIFAR100, 128, train=True)
test_loader_100 = get_dataloader(datasets.CIFAR100, 128, train=False)

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512), nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# Training and evaluation functions
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    correct, total_loss = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(dataloader), 100 * correct / len(dataloader.dataset)

# Training loop
def run_training(model_cls, train_loader, test_loader, num_classes, epochs, lr, device):
    model = model_cls(num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%")

# Set device and execute
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Training on CIFAR-10:")
run_training(SimpleCNN, train_loader_10, test_loader_10, num_classes=10, epochs=10, lr=0.001, device=device)

print("Training on CIFAR-100:")
run_training(SimpleCNN, train_loader_100, test_loader_100, num_classes=100, epochs=10, lr=0.001, device=device)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Training on CIFAR-10:
Epoch 1/10: Train Loss: 1.5162, Test Loss: 1.2223, Accuracy: 55.10%
Epoch 2/10: Train Loss: 1.0541, Test Loss: 0.9531, Accuracy: 66.23%
